## CHOPCHOP

In [35]:
import os
import itertools
import subprocess

In [31]:
def clean_path(directory, extension):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if not filename.endswith(extension):
                os.remove(file_path)

In [19]:
cmd = f"-J -BED -G hg38 -filterGCmin 30 -filterGCmax 70 -t WHOLE -n N -R 4 " + \
       "-3 PRODUCT_SIZE_MIN=150,PRODUCT_SIZE_MAX=290,PRIMER_MIN_SIZE=18,PRIMER_MAX_SIZE=25,PRIMER_OPT_SIZE=22,PRIMER_MIN_TM=57,PRIMER_MAX_TM=63,PRIMER_OPT_TM=60 " + \
       "-A 290 -DF 300 -a 20 -T 1 -g 20 -scoringMethod DOENCH_2014 -f NN -v 3 -M NGG -BB AGGCTAGTCCGT"

cmd_user   = " ".join( [f"--targets chr11:5290000-5292000 -o ./result",cmd] )

In [26]:
os.makedirs('./result', exist_ok=True)

with open('./result/_result_.tsv', 'w') as f:
    subprocess.run(
        ["python", "./chopchop/chopchop_py3.py"] + cmd_user.split(),
        stdout=f,
        text=True
    )
clean_path('./result','.tsv')

/cluster2/huanglab/liquan/miniconda3/envs/onlyC/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


In [30]:
table = pd.read_csv(f'./result/_result_.tsv',sep='\t',skiprows=1)
table = table[(table['MM0']==0) & (table['MM1']<=1)].reset_index(drop=True)
table[['chr','start']] = table['Genomic location'].str.split(':',expand=True).values
table['start'] = table['start'].astype(int);table['end'] = table['start']+23
table.head(n=10)

,Rank,Target sequence,Genomic location,Strand,GC content (%),Self-complementarity,MM0,MM1,MM2,MM3,Efficiency,chr,start,end
0,1,TCGTCATAATATGGGTTTTATGG,chr11:5291785,-,30,0,0,0,0,1,0.02,chr11,5291785,5291808
1,2,AGTTTAGGCTAGTAAGCATGAGG,chr11:5291658,-,40,4,0,0,0,3,0.41,chr11,5291658,5291681
2,3,CTGCTGTTATGACCACTAGAGGG,chr11:5291438,-,45,1,0,0,0,4,0.78,chr11,5291438,5291461
3,4,GAGTCTTGCAGGGTAGTGTAGGG,chr11:5290281,-,50,0,0,0,0,4,0.34,chr11,5290281,5290304
4,5,CGTGAAGCACGTCCAAGTGAAGG,chr11:5291862,+,55,1,0,0,0,4,0.34,chr11,5291862,5291885
5,6,CAATAGGGTCATGTTCAGTAGGG,chr11:5290409,+,40,0,0,0,0,4,0.32,chr11,5290409,5290432
6,7,ACGTAGTCTGTGCTTATGACTGG,chr11:5291842,-,45,2,0,0,0,4,0.20,chr11,5291842,5291865
7,8,ACAATAGGGTCATGTTCAGTAGG,chr11:5290408,+,40,0,0,0,0,4,0.03,chr11,5290408,5290431
8,9,CAAGGTACTTTGGTCTGGACAGG,chr11:5290032,+,50,0,0,0,1,3,0.05,chr11,5290032,5290055
9,10,GATGTCCTGTCCCTGTAAGGTGG,chr11:5291324,+,55,0,0,0,0,5,0.39,chr11,5291324,5291347


In [39]:
permutations = list(itertools.permutations(table.index, 2))
pair_df = pd.DataFrame({
    'chr':           [table.loc[i, 'chr'] for i, j in permutations],
    'Cutsite_1':     [table.loc[i, 'start'] for i, j in permutations],
    'Cutsite_2':     [table.loc[j, 'start'] for i, j in permutations],
    'seq1':          [table.loc[i, 'Target sequence'] for i, j in permutations],
    'seq2':          [table.loc[j, 'Target sequence'] for i, j in permutations],
})
pair_df['pair_length'] = pair_df['Cutsite_2'].astype(int) - pair_df['Cutsite_1'].astype(int)
pair_df_filtered = pair_df[(pair_df['pair_length'] >= 50) & (pair_df['pair_length'] <= 200)].reset_index(drop=True)
pair_df_filtered.columns=['chr','start','end','seq1','seq2','length']
pair_df_filtered.head(n=10)

,chr,start,end,seq1,seq2,length
0,chr11,5291785,5291862,TCGTCATAATATGGGTTTTATGG,CGTGAAGCACGTCCAAGTGAAGG,77
1,chr11,5291785,5291842,TCGTCATAATATGGGTTTTATGG,ACGTAGTCTGTGCTTATGACTGG,57
2,chr11,5291785,5291866,TCGTCATAATATGGGTTTTATGG,AAGCACGTCCAAGTGAAGGCAGG,81
3,chr11,5291785,5291985,TCGTCATAATATGGGTTTTATGG,ACAGATAACTCTGCTAATAAAGG,200
4,chr11,5291785,5291916,TCGTCATAATATGGGTTTTATGG,TTCTGCTATTCTTGATCAAATGG,131
5,chr11,5291785,5291977,TCGTCATAATATGGGTTTTATGG,AGCAGAGTTATCTGTACTGTTGG,192
6,chr11,5291785,5291921,TCGTCATAATATGGGTTTTATGG,TGATCAAGAATAGCAGAAAAAGG,136
7,chr11,5291785,5291874,TCGTCATAATATGGGTTTTATGG,TCATTTCTCCTGCCTTCACTTGG,89
8,chr11,5291785,5291878,TCGTCATAATATGGGTTTTATGG,GTGAAGGCAGGAGAAATGAGAGG,93
9,chr11,5291785,5291891,TCGTCATAATATGGGTTTTATGG,AAATGAGAGGAGCAAGAAAGAGG,106


## Sequence alignments & Epigenomic feature

In [1]:
import numpy as np
import pandas as pd

import pyBigWig
from pyfaidx import Fasta

In [2]:
trans_base_dict={'A':'T','C':'G','G':'C','T':'A','N':'N'}
chr_list = [f'chr{i}' for i in range(1,23)]+['chrX']

def reverse_DNA(seq):
    ### 用于反转DNA
    return ''.join([trans_base_dict[c] for c in seq])[::-1]

def gc_content(seq_list):
    ### 计算GC含量
    content = []
    for seq in seq_list:
        if len(seq)==0:
            content.append(0.0)
        else:
            content.append( (seq.upper().count('C')+seq.upper().count('G')) / len(seq) )
    return content

def find_contextual_single(data,fa=None,col_name = 'sequence',expand_flank=30,upstream=4,spacer_len=20,downstream=6):
    
    # 自动寻找上下文并且把补全上下文之后的序列整理成有30bp的格式
    ## 因为参考基因组并不会随着细胞系的改变而改变，所以这里可以这么做

    p1,p2=[],[];e1,e2=[],[]
    adj_s1,adj_s2=[],[]
    merge_table=data
    
    for _,row in merge_table.iterrows():
    
        s1 = row[col_name].upper().replace(' ','') # 因为输出是ChopChop的,非常稳定，就是20 bp protospacer + 3 bp PAM

        try:
            c,s,e = row['chr'],int(row['start']),int(row['end'])
            if not c in chr_list:
                p1.append('-1');e1.append('N'*total_len);adj_s1.append(-1)
                continue
        except:
            p1.append('-1');e1.append('N'*upstream+s1+'N'*downstream);adj_s1.append(-1)
            continue

        sequence=fa[c][s-expand_flank:e+expand_flank].seq.upper()

        l1=sequence.find(s1)

        if l1<0: # 没有抓取到位置信息的情况,自动认为是负向的
            l1=sequence.find(reverse_DNA(s1))
            p1.append('0');e1.append( reverse_DNA( fa[c][s-expand_flank+l1-downstream:s-expand_flank+l1+spacer_len+upstream].seq.upper() ) )
        else:
            p1.append('1');e1.append( fa[c][s-expand_flank+l1-upstream:s-expand_flank+l1+spacer_len+downstream].seq.upper() )
        adj_s1.append(s-expand_flank+l1)
    
    return p1,e1,adj_s1

def gain_epivalue_TPM(region,bw=None):
    
    chromo=region[0]
    start=min(region[1],region[2])
    end=max(region[1],region[2])
    
    if start==end:
        end+=1
        
    delta=end-start
    
    try:
        values = bw.stats(chromo, start, end, type='sum')[0]
        tpm = (values/delta)/(bw.header()['sumData']/bw.header()['nBasesCovered'])*1000
    except Exception as e:
        tpm = 0
    return tpm

def gain_series_TPM(data,epi='DNase',cellline='A549',genome='hg38',lo=None):
    
    epi_value_series=[]
    
    if genome=='hg38': 
        bw=pyBigWig.open(f"epigenome_ref/{cellline}/{epi}.bigWig")
    elif genome=='hg19':
        bw=pyBigWig.open(f"epigenome_ref/{cellline}/hg19/{epi}.bigWig")
    
    for i in range(len(data)):
        c,s,e = data.at[i,'chr'],data.at[i,'start'],data.at[i,'end']
        if s==e:
            e+=1
        else:
            s,e = min(s,e),max(s,e)
        if not lo is None:
            c,s,e = convert_loci([c,s,e],lo=lo)
        
        try:
            epi_value_series.append(gain_epivalue_TPM([c,s,e],bw=bw))
        except:
            epi_value_series.append(0.0)
    
    bw.close()
    
    return epi_value_series

In [40]:
test_data = pair_df_filtered.copy()

In [41]:
test_data.head(n=10)

,chr,start,end,seq1,seq2,length
0,chr11,5291785,5291862,TCGTCATAATATGGGTTTTATGG,CGTGAAGCACGTCCAAGTGAAGG,77
1,chr11,5291785,5291842,TCGTCATAATATGGGTTTTATGG,ACGTAGTCTGTGCTTATGACTGG,57
2,chr11,5291785,5291866,TCGTCATAATATGGGTTTTATGG,AAGCACGTCCAAGTGAAGGCAGG,81
3,chr11,5291785,5291985,TCGTCATAATATGGGTTTTATGG,ACAGATAACTCTGCTAATAAAGG,200
4,chr11,5291785,5291916,TCGTCATAATATGGGTTTTATGG,TTCTGCTATTCTTGATCAAATGG,131
5,chr11,5291785,5291977,TCGTCATAATATGGGTTTTATGG,AGCAGAGTTATCTGTACTGTTGG,192
6,chr11,5291785,5291921,TCGTCATAATATGGGTTTTATGG,TGATCAAGAATAGCAGAAAAAGG,136
7,chr11,5291785,5291874,TCGTCATAATATGGGTTTTATGG,TCATTTCTCCTGCCTTCACTTGG,89
8,chr11,5291785,5291878,TCGTCATAATATGGGTTTTATGG,GTGAAGGCAGGAGAAATGAGAGG,93
9,chr11,5291785,5291891,TCGTCATAATATGGGTTTTATGG,AAATGAGAGGAGCAAGAAAGAGG,106


In [44]:
with Fasta("genome_ref/hg38.fa") as hg38:
    _,test_data['seq1'],_ = find_contextual_single(test_data,col_name='seq1',fa=hg38,upstream=4,spacer_len=23,downstream=3)
    _,test_data['seq2'],_ = find_contextual_single(test_data,col_name='seq2',fa=hg38,upstream=4,spacer_len=23,downstream=3)
    test_data['Median_sequence']=test_data.apply( lambda x: hg38[x['chr']][ x['start']:x['end'] ].seq.upper() , axis=1 )
test_data['GC']=gc_content(test_data['Median_sequence'])
test_data['length'] = abs(test_data['end'] - test_data['start'])

In [45]:
test_data['DNase']=gain_series_TPM( test_data, epi='DNase'  , cellline='A549', genome='hg38', lo=None)
test_data['ATAC'] =gain_series_TPM( test_data, epi='ATAC'   , cellline='A549', genome='hg38', lo=None)
test_data['H3K27ac']=gain_series_TPM( test_data, epi='H3K27ac', cellline='A549', genome='hg38', lo=None)
test_data['H3K4me3']=gain_series_TPM( test_data, epi='H3K4me3', cellline='A549', genome='hg38', lo=None)

In [46]:
test_data.head(n=10)

,chr,start,end,seq1,seq2,length,Median_sequence,GC,DNase,ATAC,H3K27ac,H3K4me3
0,chr11,5291785,5291862,GTCATAATATGGGTTTTATGGTCCATTATT,ACAGACTACGTGAAGCACGTCCAAGTGAAG,77,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.324675,454.662252,2853.603889,1322.459159,550.255861
1,chr11,5291785,5291842,GTCATAATATGGGTTTTATGGTCCATTATT,GTAGTCTGTGCTTATGACTGGATTAAAAAT,57,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.280702,468.210673,2688.101304,1384.461209,493.102966
2,chr11,5291785,5291866,GTCATAATATGGGTTTTATGGTCCATTATT,ACTACGTGAAGCACGTCCAAGTGAAGGCAG,81,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.333333,457.265412,35.065233,1314.847333,557.504731
3,chr11,5291785,5291985,GTCATAATATGGGTTTTATGGTCCATTATT,CCAACAGTACAGATAACTCTGCTAATAAAG,200,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.380000,446.491583,60.517150,1210.546589,579.800512
4,chr11,5291785,5291916,GTCATAATATGGGTTTTATGGTCCATTATT,CTGCTATTCTTGATCAAATGGCTCCTCTTT,131,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.404580,463.997861,47.318627,1289.165313,596.854146
5,chr11,5291785,5291977,GTCATAATATGGGTTTTATGGTCCATTATT,CAGAGTTATCTGTACTGTTGGCATGACAAT,192,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.380208,445.540262,63.038698,1223.746232,578.590174
6,chr11,5291785,5291921,GTCATAATATGGGTTTTATGGTCCATTATT,GAGCCATTTGATCAAGAATAGCAGAAAAAG,136,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.397059,461.861975,70.579040,1280.722875,595.798760
7,chr11,5291785,5291874,GTCATAATATGGGTTTTATGGTCCATTATT,ATTTCTCCTGCCTTCACTTGGACGTGCTTC,89,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.359551,461.769756,31.913302,1297.839873,571.119849
8,chr11,5291785,5291878,GTCATAATATGGGTTTTATGGTCCATTATT,ACGTCCAAGTGAAGGCAGGAGAAATGAGAG,93,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.365591,463.731326,30.540687,1293.666140,577.305520
9,chr11,5291785,5291891,GTCATAATATGGGTTTTATGGTCCATTATT,GGCAGGAGAAATGAGAGGAGCAAGAAAGAG,106,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.386792,469.084004,58.478681,1288.960167,596.110285


## Single Guide Prediction

In [8]:
import utils.DeepSpCas9_main as DeepSpCas9
import utils.DeepCRISPR_main as DeepCRISPR
import utils.DistillatedRuleSet2 as Ruleset2
import utils.DistillatedCRISPRedict as CRISPRedict

2025-10-16 11:22:56.173108: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcudart.so.12
/cluster2/huanglab/liquan/miniconda3/envs/cp38tf1/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


TensorFlow version: 1.15.5
Running in TensorFlow 1.x compatibility mode.



In [47]:
test_data['DeepSpCas9_s1'] = DeepSpCas9.predict_sequence(test_data['seq1'])
test_data['DeepSpCas9_s2'] = DeepSpCas9.predict_sequence(test_data['seq2'])
test_data['DeepCRISPR_s1'] = DeepCRISPR.predict_sequence([seq[4:27] for seq in test_data['seq1']])
test_data['DeepCRISPR_s2'] = DeepCRISPR.predict_sequence([seq[4:27] for seq in test_data['seq2']])
test_data['Ruleset2_s1']   = Ruleset2.predict(test_data['seq1'],model_path='models/DistillatedRuleSet2.pth')
test_data['Ruleset2_s2']   = Ruleset2.predict(test_data['seq2'],model_path='models/DistillatedRuleSet2.pth')
test_data['CRISPRedit_s1'] = CRISPRedict.predict(test_data['seq1'],model_path='models/DistillatedCRISPRedict.pth')
test_data['CRISPRedit_s2'] = CRISPRedict.predict(test_data['seq2'],model_path='models/DistillatedCRISPRedict.pth')

INFO:tensorflow:Restoring parameters from ./DeepCas9_Final/PreTrain-Final-False-3-5-7-100-70-40-0.001-550-True-80-60


INFO:tensorflow:Restoring parameters from ./DeepCas9_Final/PreTrain-Final-False-3-5-7-100-70-40-0.001-550-True-80-60


INFO:tensorflow:Restoring parameters from ./DeepCas9_Final/PreTrain-Final-False-3-5-7-100-70-40-0.001-550-True-80-60


INFO:tensorflow:Restoring parameters from ./DeepCas9_Final/PreTrain-Final-False-3-5-7-100-70-40-0.001-550-True-80-60


/cluster2/huanglab/liquan/pycode/dual/20250306_demo/DeepCRISPR_Seq/model.ckpt-seq loaded
/cluster2/huanglab/liquan/pycode/dual/20250306_demo/DeepCRISPR_Seq/model.ckpt-seq loaded


In [48]:
def norm(x):
    return (x-x.min())/(x.max()-x.min())

if not test_data['DeepSpCas9_s1'].max() - test_data['DeepSpCas9_s1'].min() == 0:
    test_data['DeepSpCas9_s1'] = norm(test_data['DeepSpCas9_s1'])*10
if not test_data['DeepSpCas9_s2'].max() - test_data['DeepSpCas9_s2'].min() == 0:
    test_data['DeepSpCas9_s2'] = norm(test_data['DeepSpCas9_s2'])*10
    
if not test_data['DeepCRISPR_s1'].max() - test_data['DeepCRISPR_s1'].min() == 0:
    test_data['DeepSpCas9_s1'] = norm(test_data['DeepCRISPR_s1'])*10
if not test_data['DeepCRISPR_s2'].max() - test_data['DeepCRISPR_s2'].min() == 0:
    test_data['DeepSpCas9_s2'] = norm(test_data['DeepCRISPR_s2'])*10

if not test_data['CRISPRedit_s1'].max() - test_data['CRISPRedit_s1'].min() == 0:
    test_data['CRISPRedit_s1'] = norm(test_data['CRISPRedit_s1'])*10
if not test_data['CRISPRedit_s2'].max() - test_data['CRISPRedit_s2'].min() == 0:
    test_data['CRISPRedit_s2'] = norm(test_data['CRISPRedit_s2'])*10

if not test_data['Ruleset2_s1'].max() - test_data['Ruleset2_s1'].min() == 0:
    test_data['Ruleset2_s1'] = norm(test_data['Ruleset2_s1'])*10
if not test_data['Ruleset2_s2'].max() - test_data['Ruleset2_s2'].min() == 0:
    test_data['Ruleset2_s2'] = norm(test_data['Ruleset2_s2'])*10

In [49]:
test_data.head(n=10)

,chr,start,end,seq1,seq2,length,Median_sequence,GC,DNase,ATAC,H3K27ac,H3K4me3,DeepSpCas9_s1,DeepSpCas9_s2,DeepCRISPR_s1,DeepCRISPR_s2,Ruleset2_s1,Ruleset2_s2,CRISPRedit_s1,CRISPRedit_s2
0,chr11,5291785,5291862,GTCATAATATGGGTTTTATGGTCCATTATT,ACAGACTACGTGAAGCACGTCCAAGTGAAG,77,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.324675,454.662252,2853.603889,1322.459159,550.255861,3.19893,4.256242,0.192155,0.231840,6.068326,5.186392,2.192548,7.506630
1,chr11,5291785,5291842,GTCATAATATGGGTTTTATGGTCCATTATT,GTAGTCTGTGCTTATGACTGGATTAAAAAT,57,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.280702,468.210673,2688.101304,1384.461209,493.102966,3.19893,6.179154,0.192155,0.304014,6.068326,4.067711,2.192548,2.174820
2,chr11,5291785,5291866,GTCATAATATGGGTTTTATGGTCCATTATT,ACTACGTGAAGCACGTCCAAGTGAAGGCAG,81,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.333333,457.265412,35.065233,1314.847333,557.504731,3.19893,1.891667,0.192155,0.143088,6.068326,4.573349,2.192548,7.530055
3,chr11,5291785,5291985,GTCATAATATGGGTTTTATGGTCCATTATT,CCAACAGTACAGATAACTCTGCTAATAAAG,200,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.380000,446.491583,60.517150,1210.546589,579.800512,3.19893,5.126798,0.192155,0.264515,6.068326,6.446852,2.192548,7.846639
4,chr11,5291785,5291916,GTCATAATATGGGTTTTATGGTCCATTATT,CTGCTATTCTTGATCAAATGGCTCCTCTTT,131,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.404580,463.997861,47.318627,1289.165313,596.854146,3.19893,2.831964,0.192155,0.178381,6.068326,6.697123,2.192548,3.600533
5,chr11,5291785,5291977,GTCATAATATGGGTTTTATGGTCCATTATT,CAGAGTTATCTGTACTGTTGGCATGACAAT,192,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.380208,445.540262,63.038698,1223.746232,578.590174,3.19893,1.880478,0.192155,0.142668,6.068326,4.290374,2.192548,7.870648
6,chr11,5291785,5291921,GTCATAATATGGGTTTTATGGTCCATTATT,GAGCCATTTGATCAAGAATAGCAGAAAAAG,136,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.397059,461.861975,70.579040,1280.722875,595.798760,3.19893,4.648728,0.192155,0.246571,6.068326,5.610874,2.192548,8.035863
7,chr11,5291785,5291874,GTCATAATATGGGTTTTATGGTCCATTATT,ATTTCTCCTGCCTTCACTTGGACGTGCTTC,89,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.359551,461.769756,31.913302,1297.839873,571.119849,3.19893,6.959639,0.192155,0.333309,6.068326,2.668891,2.192548,5.383963
8,chr11,5291785,5291878,GTCATAATATGGGTTTTATGGTCCATTATT,ACGTCCAAGTGAAGGCAGGAGAAATGAGAG,93,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.365591,463.731326,30.540687,1293.666140,577.305520,3.19893,3.347666,0.192155,0.197738,6.068326,6.062127,2.192548,3.473499
9,chr11,5291785,5291891,GTCATAATATGGGTTTTATGGTCCATTATT,GGCAGGAGAAATGAGAGGAGCAAGAAAGAG,106,CATAAAACCCATATTATGACGAACAACATTAGGATAAGTCCATATC...,0.386792,469.084004,58.478681,1288.960167,596.110285,3.19893,1.257963,0.192155,0.119303,6.068326,4.908872,2.192548,7.660681


## HyenaDNA Prediction

In [12]:
import utils.HyenaDNA_main as Hyena

Loaded pretrained weights ok!


In [50]:
test_data['DeepSpCas9_harmony'] = 1/(1/(test_data['DeepSpCas9_s1']+1)+1/(test_data['DeepSpCas9_s2']+1))
test_data['DeepCRISPR_harmony'] = 1/(1/(test_data['DeepCRISPR_s1']+1)+1/(test_data['DeepCRISPR_s2']+1))
test_data['CRISPRedict_harmony']= 1/(1/(test_data['CRISPRedit_s1']+1)+1/(test_data['CRISPRedit_s2']+1))
test_data['Ruleset2_harmony']   = 1/(1/(test_data['Ruleset2_s1']+1)+1/(test_data['Ruleset2_s2']+1))
test_data = test_data.sort_values(
    by=['DeepSpCas9_harmony','DeepCRISPR_harmony','CRISPRedict_harmony','Ruleset2_harmony','DNase','ATAC','H3K27ac','H3K4me3'],
    ascending=False
)
test_data['hyena_score']=np.array([Hyena.hyena_inference(s[:16384]) for s in list(test_data['Median_sequence'].fillna('N'))])

In [51]:
(
test_data
.drop(['DeepSpCas9_s1','DeepSpCas9_s2','DeepCRISPR_s1','DeepCRISPR_s2',
       'CRISPRedit_s1','CRISPRedit_s2','Ruleset2_s1','Ruleset2_s2'],axis=1)
).to_csv('data/test_data_annot.csv',index=False)

In [52]:
test_data.head(n=10)

,chr,start,end,seq1,seq2,length,Median_sequence,GC,DNase,ATAC,...,DeepCRISPR_s2,Ruleset2_s1,Ruleset2_s2,CRISPRedit_s1,CRISPRedit_s2,DeepSpCas9_harmony,DeepCRISPR_harmony,CRISPRedict_harmony,Ruleset2_harmony,hyena_score
275,chr11,5291488,5291613,TATTTCCTGACCTATATCTGGCAGGACTCT,CTCAGGTTATTCTGTGACCAACAGACTGTG,125,CAGATATAGGTCAGGAAATATAATCCACTAATAAAAAGAGAAACAT...,0.304000,8208.130746,159.455269,...,0.423448,5.177202,5.992701,6.553412,7.758613,4.912534,0.702027,4.055745,3.279851,1.507686
540,chr11,5291613,5291755,CTCAGGTTATTCTGTGACCAACAGACTGTG,ATTTCCTATGCATTGATCTGGAGAAGGCTT,142,TTCTGTGACCAACAGACTGTGGGAAAAATCAGAGAAGGAGGCATCC...,0.408451,392.323763,81.144439,...,0.376697,5.992702,5.260906,7.758613,2.519409,4.849280,0.699841,2.510595,3.303301,-0.793135
281,chr11,5291488,5291632,TATTTCCTGACCTATATCTGGCAGGACTCT,AACAGACTGTGGGAAAAATCAGAGAAGGAG,144,CAGATATAGGTCAGGAAATATAATCCACTAATAAAAAGAGAAACAT...,0.326389,7193.468438,138.416032,...,0.404822,5.177202,5.396271,6.553412,6.347191,4.798094,0.697467,3.724437,3.142414,1.592556
776,chr11,5290902,5290991,GAAGAAGTGATATTGAGAAGGTAGGGTTGC,AATGAGAGAGAGAAATGGGGAGGGTAGAAG,89,CTTCTCAATATCACTTCTTCACTTAGAAAAAACCAGCCTTAGCTGT...,0.348315,114.017338,16.464842,...,0.437381,9.996129,3.593616,8.641760,6.422892,4.763632,0.697765,4.194035,3.240079,6.968378
869,chr11,5291632,5291755,AACAGACTGTGGGAAAAATCAGAGAAGGAG,ATTTCCTATGCATTGATCTGGAGAAGGCTT,123,TGGGAAAAATCAGAGAAGGAGGCATCCTCATGCTTACTAGCCTAAA...,0.398374,372.901325,93.678946,...,0.376697,5.396274,5.260906,6.347191,2.519409,4.737735,0.695309,2.379564,3.163933,1.433159
779,chr11,5290902,5291029,GAAGAAGTGATATTGAGAAGGTAGGGTTGC,ATAGGGTAAGAGACAGGGAAGGAGGTGTGG,127,CTTCTCAATATCACTTCTTCACTTAGAAAAAACCAGCCTTAGCTGT...,0.385827,140.627320,29.969684,...,0.429721,9.996129,5.288286,8.641760,9.191180,4.722998,0.695955,4.954430,4.000529,7.430711
785,chr11,5290902,5291023,GAAGAAGTGATATTGAGAAGGTAGGGTTGC,AGGAAGATAGGGTAAGAGACAGGGAAGGAG,121,CTTCTCAATATCACTTCTTCACTTAGAAAAAACCAGCCTTAGCTGT...,0.380165,137.536905,31.455784,...,0.425565,9.996129,5.776807,8.641760,8.766798,4.700585,0.694969,4.851938,4.192816,7.309916
290,chr11,5290264,5290353,AGGGACCACAGGGTTAAGGGGGCAGTAGAA,GGATGTTGAAAGACAGAGAGGATGGGGTGC,89,CCCTTAACCCTGTGGTCCCTACACTACCCTGCAAGACTCTTAGCTG...,0.516854,18.242779,25.437811,...,0.376968,3.604665,3.776971,5.403442,8.378635,4.647241,0.691752,3.805300,2.344618,-7.176785
284,chr11,5290264,5290453,AGGGACCACAGGGTTAAGGGGGCAGTAGAA,AAGGGTAAACTCAAACTATGGCTTGTCTAA,189,CCCTTAACCCTGTGGTCCCTACACTACCCTGCAAGACTCTTAGCTG...,0.402116,8.590515,33.000743,...,0.366581,3.604664,9.133747,5.403442,4.647042,4.574344,0.689120,3.000751,3.166047,-5.737172
304,chr11,5291375,5291488,AAGGGCTCCTTAACAACTTCCTTGCTTGGG,TATTTCCTGACCTATATCTGGCAGGACTCT,113,TTAACAACTTCCTTGCTTGGGGCTCCACCATCTTGGACCATTAGCT...,0.486726,31178.972961,677.976687,...,0.385183,5.512974,5.177200,6.239966,6.553412,4.568961,0.688829,3.696684,3.170322,-2.534668
